In [1]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %                   SCF-TB - PROXY APPLICATION                      %
# %                   A.M.N. Niklasson, M. Kulichenko. T1, LANL       %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# % Total Energy Function:                                            %
# % E = 2Tr[H0(D-D0)] + (1/2)*sum_i U_i q_i^2 +                       %
# %      + (1/2)sum_{i,j (i!=j)} q_i C_{ij} q_j - Efield*dipole       %
# % dipole = sum_i R_{i} q_i                                          %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

import torch
import warnings
import logging
import os
# to disable torchdynamo completely. Faster for smaller systems and single-point calculations.
#os.environ["TORCHDYNAMO_DISABLE"] = "1"  # hard-disable capture

import numpy as np
import matplotlib.pyplot as plt

from dftorch.SCF import SCFx, SCFx_batch, scf_x_os
from dftorch.Spin import get_spin_energy, get_h_spin
from dftorch.Energy import Energy

from dftorch.Forces import Forces, forces_spin

from dftorch.Constants import Constants
from dftorch.Structure import Structure, StructureBatch
from dftorch.MD import MDXL, MDXLBatch, MDXLOS
from dftorch.ESDriver import ESDriver, ESDriverBatch

### Configure torch and torch.compile ###
# Silence warnings and module logs
warnings.filterwarnings("ignore")
os.environ["TORCH_LOGS"] = ""               # disable PT2 logging
os.environ["TORCHINDUCTOR_VERBOSE"] = "0"
os.environ["TORCHDYNAMO_VERBOSE"] = "0"
logging.getLogger("torch.fx").setLevel(logging.CRITICAL)
logging.getLogger("torch.fx.experimental.symbolic_shapes").setLevel(logging.CRITICAL)
logging.getLogger("torch.fx.experimental.recording").setLevel(logging.CRITICAL)
# Enable dynamic shape capture for dynamo
torch._dynamo.config.capture_dynamic_output_shape_ops = True
# default data type
torch.set_default_dtype(torch.float64)

torch.cuda.empty_cache()

# 1. MD with PBC and PME
### Note, PME does not work properly for small cells (< 25x25x25)
### Also, do not use PME without PBC.
### Charged systems with PBE is a bad idea.
## First, SCF and forces

In [3]:
%%time
dftorch_params = {
    'coul_method': 'PME',  # 'FULL' for full coulomb matrix, 'PME' for PME method
    'Coulomb_acc': 5e-5,   # Coulomb accuracy for full coulomb calcs or t_err for PME
    'cutoff': 12.0,        # Coulomb cutoff
    'PME_order': 4,        # Ignored for FULL coulomb method

    'SCF_MAX_ITER': 100,   # Maximum number of SCF iterations
    'SCF_TOL': 1e-6,       # SCF convergence tolerance on density matrix
    'SCF_ALPHA': 0.04,     # Scaled delta function coefficient. Acts as linear mixing coefficient used before Krylov acceleration starts.

    'KRYLOV_MAXRANK': 15,  # Maximum Krylov subspace rank
    'KRYLOV_TOL': 1e-6,    # Krylov subspace convergence tolerance in SCF
    'KRYLOV_TOL_MD': 1e-4, # Krylov subspace convergence tolerance in MD SCF
    'KRYLOV_START': 30,    # Number of initial SCF iterations before starting Krylov acceleration
                }
                
# Initial data, load atoms and coordinates, etc in COORD.dat
device = 'cuda' if torch.cuda.is_available() else 'cpu'

filename = '8RNT.xyz'      
LBox = torch.tensor([45.0, 45.0, 40.0], device=device) # Simulation box size in Angstroms. Only cubic boxes supported for now.

# filename = 'solvated_ac_2k_react.xyz'            # Solvated acetylacetone and glycine molecules in H20, Na, Cl
# LBox = torch.tensor([29.0, 29.0, 29.0], device=device) # Simulation box size in Angstroms. Only cubic boxes supported for now.

# filename = 'C840.xyz'            # Solvated acetylacetone and glycine molecules in H20, Na, Cl
# LBox = torch.tensor([32.0, 32.0, 32.0], device=device) # Simulation box size in Angstroms. Only cubic boxes supported for now.

# Create constants container. Set path to SKF files.
const = Constants(filename,
                  #'C:\\000_MyFiles\\Programs\\DFTorch\\tests\\sk_orig\\ptbp\\complete_set\\',
                  #'C:\\000_MyFiles\\Programs\\DFTorch\\tests\\sk_orig\\mio-1-1\\mio-1-1\\',
                    '/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/',

                  magnetic_hubbard_ldep=False # atom/shell-resolved spin parameters w for open-shell. 
                  ).to(device)

# Create structure object. Define total charge and electronic temperature.
structure1 = Structure(filename, LBox, const, charge=7, Te=1500.0, device=device)

# Create ESDriver object and run SCF calculation
# electronic_rcut and repulsive_rcut are in Angstroms.
# They should be >= cutoffs defined in SKF files for the element pair with largest cutoff present in the system.
es_driver = ESDriver(dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device)
es_driver(structure1, const, do_scf=True)
es_driver.calc_forces(structure1, const) # Calculate forces after SCF

int64
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/H-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/H-C.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/H-N.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/H-O.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/H-S.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/H-Zn.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/C-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/C-C.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/C-N.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/C-O.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/C-S.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/C-Zn.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1/N-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/

In [ ]:
#torch.save(structure1.f_tot, 'f_pme.pt')

In [ ]:
#charges:
structure1.q

tensor([-0.2438, -0.7706, -0.4516, -0.3950, -0.7243,  0.2517, -0.4235, -0.4312,
        -0.3921, -0.4288,  0.2427, -0.3826, -0.4541, -0.4357, -0.4114, -0.2073,
        -0.3938, -0.7813, -0.7546, -0.4127])

In [4]:
#MO coeffs
structure1.Q

tensor([[ 1.7793e-01, -3.5631e-01,  4.9032e-01,  ..., -2.9076e-01,
          3.5272e-01, -2.0136e-01],
        [-6.8824e-02,  8.9261e-02, -1.3560e-02,  ..., -1.9987e-01,
          8.2232e-02,  1.5840e-02],
        [-2.2904e-04,  1.4454e-03, -2.4637e-04,  ..., -8.8955e-02,
         -2.0499e-02,  5.5811e-02],
        ...,
        [-3.3998e-02, -8.0351e-02, -1.3895e-01,  ..., -1.3726e-01,
         -1.1252e-01, -6.3736e-02],
        [-3.9725e-02, -9.2969e-02, -1.5810e-01,  ..., -2.6449e-01,
         -2.3171e-01, -1.4924e-01],
        [-1.4647e-01, -2.3625e-01, -2.1485e-01,  ..., -2.5871e-01,
         -3.5329e-01, -3.0009e-01]])

In [5]:
print("Total Energy: ", structure1.e_tot)
print("Chemical Potential: ", structure1.mu0)

Total Energy:  tensor(-138.3300)
Chemical Potential:  tensor(-44.0957)


## MD

In [6]:
torch.manual_seed(0)
temperature_K = torch.tensor(1.0, device=structure1.device)
mdDriver = MDXL(es_driver, const, temperature_K=temperature_K)
# Set number of steps, time step (fs), dump interval and trajectory filename
mdDriver.run(structure1, dftorch_params, num_steps=20, dt=0.3, dump_interval=5, traj_filename='md_trj.xyz')

########## Step = 0 ##########
H0: 0.019 s
H1: 0.081 s
  rank: 0, Fel = 0.000164
  rank: 1, Fel = 0.000130
  rank: 2, Fel = 0.000019
KER: 0.183 s
F AND E: 0.001 s
ETOT = -138.32770692, EPOT = -138.54762254, EKIN = 0.00232668, T = 0.89999886, ResErr = 0.000000, t = 0.3 s
0.0 GB


########## Step = 1 ##########
H0: 0.018 s
H1: 0.008 s
  rank: 0, Fel = 0.000355
  rank: 1, Fel = 0.000250
  rank: 2, Fel = 0.000024
KER: 0.018 s
F AND E: 0.001 s
ETOT = -138.32879379, EPOT = -139.16758910, EKIN = 0.21882875, T = 84.64663830, ResErr = 0.001296, t = 0.0 s
0.0 GB


########## Step = 2 ##########
H0: 0.018 s
H1: 0.012 s
  rank: 0, Fel = 0.000228
  rank: 1, Fel = 0.000190
  rank: 2, Fel = 0.000013
KER: 0.189 s
F AND E: 0.001 s
ETOT = -138.33023131, EPOT = -140.13703110, EKIN = 0.83735779, T = 323.90406995, ResErr = 0.002632, t = 0.2 s
0.0 GB


########## Step = 3 ##########
H0: 0.020 s
H1: 0.070 s
  rank: 0, Fel = 0.000032
KER: 0.005 s
F AND E: 0.001 s
ETOT = -138.33158250, EPOT = -141.38006778, EK

# 2. MD without PBC and with full Coulomb
### Do not use PME without PBC.
### Turn on PBC if necessary
## First, SCF and forces

In [17]:
%%time
dftorch_params = {
    'coul_method': 'FULL', # 'FULL' for full coulomb matrix, 'PME' for PME method
    'Coulomb_acc': 5e-5,   # Coulomb accuracy for full coulomb calcs or t_err for PME
    'cutoff': 12.0,        # Coulomb cutoff
    'PME_order': 4,        # Ignored for FULL coulomb method

    'SCF_MAX_ITER': 100,    # Maximum number of SCF iterations
    'SCF_TOL': 5e-6,       # SCF convergence tolerance on density matrix
    'SCF_ALPHA': 0.035,      # Scaled delta function coefficient. Acts as linear mixing coefficient used before Krylov acceleration starts.

    'KRYLOV_MAXRANK': 15,  # Maximum Krylov subspace rank
    'KRYLOV_TOL': 9e-6,    # Krylov subspace convergence tolerance in SCF
    'KRYLOV_TOL_MD': 1e-4, # Krylov subspace convergence tolerance in MD SCF
    'KRYLOV_START': 30,     # Number of initial SCF iterations before starting Krylov acceleration
                }
                
# Initial data, load atoms and coordinates, etc in COORD.dat
device = 'cuda' if torch.cuda.is_available() else 'cpu'
filename = '8RNT.xyz'      
#filename = 'COORD.xyz'            # Solvated acetylacetone and glycine molecules in H20, Na, Cl
#LBox = torch.tensor([45.0, 45.0, 40.0], device=device) # Simulation box size in Angstroms. Only cubic boxes supported for now.
LBox = None
# Create constants container. Set path to SKF files.
const = Constants(filename,
                  #'C:\\000_MyFiles\\Programs\\DFTorch\\tests\\sk_orig\\ptbp\\complete_set\\',
                  'C:\\000_MyFiles\\Programs\\DFTorch\\tests\\sk_orig\\mio-1-1\\mio-1-1\\',
                  magnetic_hubbard_ldep=False # atom/shell-resolved spin parameters w for open-shell. 
                  ).to(device)

# Create structure object. Define total charge and electronic temperature.
structure1 = Structure(filename, LBox, const, charge=7, Te=600.0, device=device)

# Create ESDriver object and run SCF calculation
# electronic_rcut and repulsive_rcut are in Angstroms.
# They should be >= cutoffs defined in SKF files for the element pair with largest cutoff present in the system.
es_driver = ESDriver(dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device)
es_driver(structure1, const, do_scf=True)
es_driver.calc_forces(structure1, const) # Calculate forces after SCF

int64
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-H.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-C.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-N.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-O.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-S.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-Zn.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-H.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-C.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-N.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-O.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-S.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-Zn.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\N-H.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\N-C.skf
C:\000_MyFil

In [18]:
structure1.e_tot

tensor(-64511.2997)

In [19]:
torch.manual_seed(0)
temperature_K = torch.tensor(1.0, device=structure1.device)
mdDriver = MDXL(es_driver, const, temperature_K=temperature_K)
# Set number of steps, time step (fs), dump interval and trajectory filename
mdDriver.run(structure1, dftorch_params, num_steps=20, dt=0.3, dump_interval=5, traj_filename='md_trj.xyz')

########## Step = 0 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.1 s
H0: 9.544 s
H1: 14.977 s
  rank: 0, Fel = 0.000704
  rank: 1, Fel = 0.000611
  rank: 2, Fel = 0.000421
  rank: 3, Fel = 0.000286
  rank: 4, Fel = 0.000182
  rank: 5, Fel = 0.000122
  rank: 6, Fel = 0.000093
KER: 70.479 s
F AND E: 10.694 s
ETOT = -64511.07088021, EPOT = -64517.52188680, EKIN = 0.22879018, T = 0.99887007, ResErr = 0.000000, t = 105.8 s
0.0 GB


########## Step = 1 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.1 s
H0: 7.301 s
H1: 8.886 s
  rank: 0, Fel = 0.001503
  rank: 1, Fel = 0.001323
  rank: 2, Fel = 0.000936
  rank: 3, Fel = 0.000606
  rank: 4, Fel = 0.000350
  rank: 5, Fel = 0.000236
  rank: 6, Fel = 0.000190
  rank: 7, Fel = 0.000156
  rank: 8, Fel = 0.000123
  rank: 9, Fel = 0.000089
KER: 100.271 s
F AND E: 11.084 s
ETOT = -64511.15383170, EPOT = -64534.89571008, EKIN = 6.36805510, T = 27.80215312, ResErr = 0.000594, t = 127.5 s
0.0 GB


########## Step = 2 ##########
CoulombM

# 3. Open-Shell MD.
### Turn on PBC if necessary
## First, SCF and forces

In [ ]:
%%time
dftorch_params = {
    'UNRESTRICTED': True,
    'BROKEN_SYM': True,
    'coul_method': '!PME', # 'FULL' for full coulomb matrix, 'PME' for PME method
    'Coulomb_acc': 5e-5,   # Coulomb accuracy for full coulomb calcs or t_err for PME
    'cutoff': 12.0,        # Coulomb cutoff
    'PME_order': 4,        # Ignored for FULL coulomb method

    'SCF_MAX_ITER': 100,    # Maximum number of SCF iterations
    'SCF_TOL': 5e-6,       # SCF convergence tolerance on density matrix
    'SCF_ALPHA': 0.035,      # Scaled delta function coefficient. Acts as linear mixing coefficient used before Krylov acceleration starts.

    'KRYLOV_MAXRANK': 15,  # Maximum Krylov subspace rank
    'KRYLOV_TOL': 9e-6,    # Krylov subspace convergence tolerance in SCF
    'KRYLOV_TOL_MD': 1e-4, # Krylov subspace convergence tolerance in MD SCF
    'KRYLOV_START': 27,     # Number of initial SCF iterations before starting Krylov acceleration
                }
                
# Initial data, load atoms and coordinates, etc in COORD.dat
device = 'cuda' if torch.cuda.is_available() else 'cpu'
filename = '8RNT.xyz'
LBox = None
#LBox = torch.tensor([45.0, 45.0, 40.0], device=device) # Simulation box size in Angstroms. Only cubic boxes supported for now.
     
#filename = 'COORD.xyz'            # Solvated acetylacetone and glycine molecules in H20, Na, Cl
#filename = 'O2.xyz'            # Solvated acetylacetone and glycine molecules in H20, Na, Cl
# LBox = torch.tensor([20.0, 20.0, 20.0], device=device) # Simulation box size in Angstroms. Only cubic boxes supported for now.

# filename = 'C840.xyz'            # Solvated acetylacetone and glycine molecules in H20, Na, Cl
# LBox = torch.tensor([32.0, 32.0, 32.0], device=device) # Simulation box size in Angstroms. Only cubic boxes supported for now.
#LBox = None

# Create constants container. Set path to SKF files.
const = Constants(filename,
                  #'/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set',
                  #'C:\\000_MyFiles\\Programs\\DFTorch\\tests\\sk_orig\\ptbp\\complete_set\\',
                  'C:\\000_MyFiles\\Programs\\DFTorch\\tests\\sk_orig\\mio-1-1\\mio-1-1\\',
                  #'C:\\000_MyFiles\\Programs\\DFTorch\\tests\\sk_orig\\pbc-0-3\\',
                    magnetic_hubbard_ldep=False
                  ).to(device)

# Create structure object. Define total charge and electronic temperature.
structure1 = Structure(filename, LBox, const, charge=7, spin_pol=0, os=dftorch_params['UNRESTRICTED'],
                        Te=500.0, device=device)

# Create ESDriver object and run SCF calculation
# electronic_rcut and repulsive_rcut are in Angstroms.
# They should be >= cutoffs defined in SKF files for the element pair with largest cutoff present in the system.
es_driver = ESDriver(dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device)
es_driver(structure1, const, do_scf=True)
es_driver.calc_forces(structure1, const) # Calculate forces after SCF

int64
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-H.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-C.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-N.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-O.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-S.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-Zn.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-H.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-C.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-N.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-O.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-S.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-Zn.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\N-H.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\N-C.skf
C:\000_MyFil

In [18]:
structure1.e_tot

tensor(-177.8623)

In [19]:
structure1.q_tot_atom

tensor([-5.5511e-16, -2.4980e-15])

In [15]:
# net spin per shell
structure1.net_spin_sr

tensor([0., 0., 0., 0.])

In [16]:
# net spin (sums to a number of unpolarized electrons)
structure1.net_spin_sr.sum()

tensor(0.)

In [83]:
#forces
#(structure1.f_tot - torch.load('f_PME.pt')).abs().max()

#torch.save(structure1.f_tot, 'f_PME.pt')

## MD

In [33]:
torch.manual_seed(0)
temperature_K = torch.tensor(200.0, device=structure1.device)
mdDriver = MDXLOS(es_driver, const, temperature_K=temperature_K)
#mdDriver.NoRank = True
# Set number of steps, time step (fs), dump interval and trajectory filename
mdDriver.run(structure1, dftorch_params, num_steps=200, dt=0.2, dump_interval=5, traj_filename='md_trj.xyz')

########## Step = 0 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
H0: 0.025 s
H1: 0.003 s
  rank: 0, Fel = 0.000456
  rank: 1, Fel = 0.000198
  rank: 2, Fel = 0.000060
KER: 0.011 s
F AND E: 0.003 s
ETOT = -326.34083405, EPOT = -326.83971101, EKIN = 0.46533596, T = 179.99977182, ResErr = 0.000000, t = 0.0 s
0.0 GB


########## Step = 1 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
H0: 0.021 s
H1: 0.005 s
  rank: 0, Fel = 0.000249
  rank: 1, Fel = 0.000078
KER: 0.005 s
F AND E: 0.003 s
ETOT = -326.34082398, EPOT = -326.94217540, EKIN = 0.49888703, T = 192.97788978, ResErr = 0.000934, t = 0.0 s
0.0 GB


########## Step = 2 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
H0: 0.020 s
H1: 0.003 s
  rank: 0, Fel = 0.000569
  rank: 1, Fel = 0.000183
  rank: 2, Fel = 0.000048
KER: 0.009 s
F AND E: 0.002 s
ETOT = -326.34092594, EPOT = -327.11094005, EKIN = 0.60124946, T = 232.57339875, ResErr = 0.000601, t = 0.0 s
0.0 GB


########## Step = 3 ##########
Coulo

In [11]:
structure1.mu0

tensor([ 6.0181, -7.8240])